## class Ball()

This class encodes a ball object with properties [ID, pocket, position, velocity]

ID indicates the identity of the ball, [1,7] are one team, and [9, 15] are the other team. 8 is the eight-ball, and 0 is the cue ball

pocket indicates whether the ball is in a pocket.

p and v are both in the form of [x,y]

In [5]:
import numpy as np
import dataclasses

BALL_RADIUS = 0.06 # meters
ACCELERATION = - 0.1 # meters/second^2

@dataclasses.dataclass
class Ball():
    ID: int 
    pocket: bool
    p: list[float, float]
    v: list[float, float]
        
    def __init__(ID: int, x_0: float, y_0: float):
        self.ID = ID
        self.pocket = False
        self.p = [x_0,y_0]
        self.v = [0.0,0.0]
        
    def time_step():
        "steps forward position and velocity of ball"
        ...
    
    def collides_with():
        "returns true if ball is in collision with other ball object"
        ...
    
    def collides_with_table():
        "returns true if ball is touching either an edge or a pocket"
        ...
    
    def table_collision():
        "updates position and velocity of ball after wall/pocket interaction"
        ...

# Questions to be discussed
Should ball radius and acceleration be defined here in the ball class or should they be variables in the contructor/time_step() methods, respectively?

How should ball-to-ball collisions be handled? it makes less sense to have it as a method of the ball class because it has to modify both balls.

In [ ]:
# TESTS
